# Import

In [2]:
from bs4 import BeautifulSoup
import base64
import pandas as pd
# import tensorflow as tf
from pytesseract import pytesseract
from PIL import Image
import glob
import cv2
import numpy as np
from pathlib import Path
import os
import json
import collections

# Main

In [9]:
%cd C:\Users\User\Desktop\backup\brt\Projects\Crawing-List-of-Admitted-Schools-for-Students\
    
path_to_tesseract = Path(r'C:\Program Files\Tesseract-OCR\tesseract.exe')
pytesseract.tesseract_cmd = path_to_tesseract


for file in glob.iglob('data\department\*交叉查榜*.html'):
    # print(file)
    processing_department(file)

c:\Users\User\Desktop\backup\brt\Projects\Crawing-List-of-Admitted-Schools-for-Students
長庚大學中醫學系
長庚大學中醫學系
長庚大學人工智慧學系


## Functions

In [8]:
def processing_department(filename):
    with open(filename, 'r', encoding="utf-8") as file:
        html = file.read()
        # print(html)
        soup = BeautifulSoup(html, 'html.parser')
    
    fields = ['rank', 'exam_location', 'stuno', 'univ',  'year', 'department_name', 'select', 'dept_rank']
    df = pd.DataFrame(columns=fields)
    df1 = pd.DataFrame(columns=fields)
    df2 = pd.DataFrame(columns=fields)
    
    department_name = []
    select = []
    dept_rank = []

    str = soup.head.find_all('title')[0].text
    pos = str.find('年')
    year = str[pos-3:pos]  # str[5:8] str[5], str[6], str[7]

    str = soup.head.find_all('title')[0].text
    pos = str.find('-')
    univ = str[:pos].rstrip()
    

    dark_list = soup.body.find_all(bgcolor="#DEDEDC") # 褐色
    df1 = processing_list(univ, year, dark_list, df, department_name, select, dept_rank)
    
    white_list = soup.body.find_all(bgcolor="#FFFFFF") # 白色
    df2 = processing_list(univ, year, white_list, df, department_name, select, dept_rank)
    
    # make two df cross-interleave
    df = cross_interleave(df1, df2)

    # explode list into df
    df = df.explode(['department_name', 'select', 'dept_rank']).reset_index(drop=True)
    df = df.dropna()
    
    # save to csv
    result_dir = Path(r'.\result')
    df.to_csv(path_or_buf = result_dir / f'{univ}.csv', encoding='utf-8-sig', index=False)
    

### Functions

In [3]:
def processing_list(univ, year, color_list, df, department_name, select, dept_rank):
    rank = ''
    stuno = ''
    print(univ)
    for item in color_list:
        color_list_item_scope1 = item.children # get message under the soup.body.find_all(bgcolor="")
        count_item = 0
        exam_location = ""
        for item_scope2 in color_list_item_scope1:
            # print("i:", item_scope2," ", count_item)
            """
            count_item:3 = 正備取
            count_item:5 = number
            count_item:9 = 錄取校系
            """
            if count_item == 3:                
                try:
                    base64_str = item.find_all('img')[0]['src'].split(',')[1]
                except IndexError:
                    rank = "None"
                else:
                    rank = ''
                    img_chi = ''
                    img_digit = ''
                    img_digit_2 = ''
                    img_digit_3 = ''
                    with open(r'data/rank.png', "wb") as fh:
                        fh.write(base64.decodebytes(bytes(base64_str, 'utf-8')))
                    
                    img_dept_rank_crop = cv2.imread(r'data/rank.png')
                    # print(img_dept_rank_crop.shape)
                    if img_dept_rank_crop.shape[0] < 51:
                        if img_dept_rank_crop.shape[1] < 100:
                            # only one digit
                            
                            crop_img_chi = img_dept_rank_crop[0: 48, 0:48]
                            cv2.imwrite(f'data/dept_crop_img_chi.png', crop_img_chi)
                            path_to_image = f'data/dept_crop_img_chi.png'
                            cv2.imwrite(f'data/dept_crop_img_chi.png', thresholding_(path_to_image))
                            img_chi = pytesseract.image_to_string(Image.open(path_to_image), lang='chi_tra', config = '--oem 0 --psm 7')
                            
                            # first digit
                            crop_img_digit = img_dept_rank_crop[0:48, 48:76]
                            cv2.imwrite(f'data/dept_crop_img_digit.png', crop_img_digit)
                            path_to_image = f'data/dept_crop_img_digit.png'
                            cv2.imwrite(f'data/dept_crop_img_digit.png', thresholding_(path_to_image))
                            img_digit = pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config = '--oem 1 --psm 7')
                            
                            img_chi = img_chi.strip()
                            img_digit = img_digit.strip()
                            rank = f'{img_chi}{img_digit}'
                            
                        elif img_dept_rank_crop.shape[1] > 100 and img_dept_rank_crop.shape[1] < 120:
                            # two digits
                            
                            crop_img_chi = img_dept_rank_crop[0: 48, 0:48]
                            cv2.imwrite(f'data/dept_crop_img_chi.png', crop_img_chi)
                            path_to_image = f'data/dept_crop_img_chi.png'
                            cv2.imwrite(f'data/dept_crop_img_chi.png', thresholding_(path_to_image))
                            img_chi = pytesseract.image_to_string(Image.open(path_to_image), lang='chi_tra', config = '--oem 0 --psm 7')
                            
                            # first digit
                            crop_img_digit = img_dept_rank_crop[0:48, 48:76]
                            cv2.imwrite(f'data/dept_crop_img_digit.png', crop_img_digit)
                            path_to_image = f'data/dept_crop_img_digit.png'
                            cv2.imwrite(f'data/dept_crop_img_digit.png', thresholding_(path_to_image))
                            img_digit = pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config = '--oem 1 --psm 7')
                            
                            # second digit
                            crop_img_digit = img_dept_rank_crop[0:48, 76:104]
                            cv2.imwrite(f'data/dept_crop_img_digit_2.png', crop_img_digit)
                            path_to_image = f'data/dept_crop_img_digit_2.png'
                            cv2.imwrite(f'data/dept_crop_img_digit_2.png', thresholding_(path_to_image))
                            img_digit_2 = pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config = '--oem 1 --psm 7')
                            
                            img_chi = img_chi.strip()
                            img_digit = img_digit.strip()
                            img_digit_2 = img_digit_2.strip()
                            rank = f'{img_chi}{img_digit}{img_digit_2}'
                        
                        elif img_dept_rank_crop.shape[1] > 120:
                            # three digits
                            
                            crop_img_chi = img_dept_rank_crop[0: 48, 0:48]
                            cv2.imwrite(f'data/dept_crop_img_chi.png', crop_img_chi)
                            path_to_image = f'data/dept_crop_img_chi.png'
                            cv2.imwrite(f'data/dept_crop_img_chi.png', thresholding_(path_to_image))
                            img_chi = pytesseract.image_to_string(Image.open(path_to_image), lang='chi_tra', config = '--oem 0 --psm 7')
                            
                            # first digit
                            crop_img_digit = img_dept_rank_crop[0:48, 48:76]
                            cv2.imwrite(f'data/dept_crop_img_digit.png', crop_img_digit)
                            path_to_image = f'data/dept_crop_img_digit.png'
                            cv2.imwrite(f'data/dept_crop_img_digit.png', thresholding_(path_to_image))
                            img_digit = pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config = '--oem 1 --psm 7')
                            
                            # second digit
                            crop_img_digit = img_dept_rank_crop[0:48, 76:104]
                            cv2.imwrite(f'data/dept_crop_img_digit_2.png', crop_img_digit)
                            path_to_image = f'data/dept_crop_img_digit_2.png'
                            cv2.imwrite(f'data/dept_crop_img_digit_2.png', thresholding_(path_to_image))
                            img_digit_2 = pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config = '--oem 1 --psm 7')
                            
                            # third digit
                            crop_img_digit = img_dept_rank_crop[0:48, 104:132]
                            cv2.imwrite(f'data/dept_crop_img_digit_3.png', crop_img_digit)
                            path_to_image = f'data/dept_crop_img_digit_3.png'
                            cv2.imwrite(f'data/dept_crop_img_digit_3.png', thresholding_(path_to_image))
                            img_digit_3 = pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config = '--oem 1 --psm 7')
                            
                            img_chi = img_chi.strip()
                            img_digit = img_digit.strip()
                            img_digit_2 = img_digit_2.strip()
                            img_digit_3 = img_digit_3.strip()
                            rank = f'{img_chi}{img_digit}{img_digit_2}{img_digit_3}'
                            
                    elif img_dept_rank_crop.shape[0] > 51:
                        if img_dept_rank_crop.shape[1] < 100:
                            # only one digit
                            
                            crop_img_chi = img_dept_rank_crop[0: 52, 0:48]
                            cv2.imwrite(f'data/dept_crop_img_chi.png', crop_img_chi)
                            path_to_image = f'data/dept_crop_img_chi.png'
                            cv2.imwrite(f'data/dept_crop_img_chi.png', thresholding_(path_to_image))
                            img_chi = pytesseract.image_to_string(Image.open(path_to_image), lang='chi_tra', config = '--oem 0 --psm 7')
                            
                            # first digit
                            crop_img_digit = img_dept_rank_crop[0:52, 48:76]
                            cv2.imwrite(f'data/dept_crop_img_digit.png', crop_img_digit)
                            path_to_image = f'data/dept_crop_img_digit.png'
                            cv2.imwrite(f'data/dept_crop_img_digit.png', thresholding_(path_to_image))
                            img_digit = pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config = '--oem 1 --psm 7')
                            
                            img_chi = img_chi.strip()
                            img_digit = img_digit.strip()
                            rank = f'{img_chi}{img_digit}'
                            
                        elif img_dept_rank_crop.shape[1] > 100 and img_dept_rank_crop.shape[1] < 120:
                            # two digits
                            
                            crop_img_chi = img_dept_rank_crop[0: 52, 0:48]
                            cv2.imwrite(f'data/dept_crop_img_chi.png', crop_img_chi)
                            path_to_image = f'data/dept_crop_img_chi.png'
                            cv2.imwrite(f'data/dept_crop_img_chi.png', thresholding_(path_to_image))
                            img_chi = pytesseract.image_to_string(Image.open(path_to_image), lang='chi_tra', config = '--oem 0 --psm 7')
                            
                            # first digit
                            crop_img_digit = img_dept_rank_crop[0:52, 48:76]
                            cv2.imwrite(f'data/dept_crop_img_digit.png', crop_img_digit)
                            path_to_image = f'data/dept_crop_img_digit.png'
                            cv2.imwrite(f'data/dept_crop_img_digit.png', thresholding_(path_to_image))
                            img_digit = pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config = '--oem 1 --psm 7')
                            
                            # second digit
                            crop_img_digit = img_dept_rank_crop[0:52, 76:104]
                            cv2.imwrite(f'data/dept_crop_img_digit.png', crop_img_digit)
                            path_to_image = f'data/dept_crop_img_digit_2.png'
                            cv2.imwrite(f'data/dept_crop_img_digi_2t.png', thresholding_(path_to_image))
                            img_digit_2 = pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config = '--oem 1 --psm 7')
                            
                            img_chi = img_chi.strip()
                            img_digit = img_digit.strip()
                            img_digit_2 = img_digit_2.strip()
                            rank = f'{img_chi}{img_digit}{img_digit_2}'
                        
                        elif img_dept_rank_crop.shape[1] > 120:
                            # three digits
                            
                            crop_img_chi = img_dept_rank_crop[0: 52, 0:48]
                            cv2.imwrite(f'data/dept_crop_img_chi.png', crop_img_chi)
                            path_to_image = f'data/dept_crop_img_chi.png'
                            cv2.imwrite(f'data/dept_crop_img_chi.png', thresholding_(path_to_image))
                            img_chi = pytesseract.image_to_string(Image.open(path_to_image), lang='chi_tra', config = '--oem 0 --psm 7')
                            
                            # first digit
                            crop_img_digit = img_dept_rank_crop[0:52, 48:76]
                            cv2.imwrite(f'data/dept_crop_img_digit.png', crop_img_digit)
                            path_to_image = f'data/dept_crop_img_digit.png'
                            cv2.imwrite(f'data/dept_crop_img_digit.png', thresholding_(path_to_image))
                            img_digit = pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config = '--oem 1 --psm 7')
                            
                            # second digit
                            crop_img_digit = img_dept_rank_crop[0:52, 76:104]
                            cv2.imwrite(f'data/dept_crop_img_digit_2.png', crop_img_digit)
                            path_to_image = f'data/dept_crop_img_digit_2.png'
                            cv2.imwrite(f'data/dept_crop_img_digit_2.png', thresholding_(path_to_image))
                            img_digit_2 = pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config = '--oem 1 --psm 7')
                            
                            # third digit
                            crop_img_digit = img_dept_rank_crop[0:52, 104:132]
                            cv2.imwrite(f'data/dept_crop_img_digit_3.png', crop_img_digit)
                            path_to_image = f'data/dept_crop_img_digit_3.png'
                            cv2.imwrite(f'data/dept_crop_img_digit_3.png', thresholding_(path_to_image))
                            img_digit_3 = pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config = '--oem 1 --psm 7')
                            
                            img_chi = img_chi.strip()
                            img_digit = img_digit.strip()
                            img_digit_2 = img_digit_2.strip()
                            img_digit_3 = img_digit_3.strip()
                            rank = f'{img_chi}{img_digit}{img_digit_2}{img_digit_3}'
                                       
            elif count_item == 5:
                stuno = 'temp'

                # get exam location
                x = item.find_all('a')
                exam_location = x[0].text[6:-1]
            elif count_item == 9:
                tr_scope3 = item_scope2.find_all('tr')
                department_name = []
                select = []
                dept_rank = []
                for item_scope4 in tr_scope3:
                    # print("item_scope4:", count, rank, item_scope4)
                    count_item_scope5 = 0
                    for item_scope5 in item_scope4:
                        # print('item_scope5:', count_, item_scope5)
                        """_summary_
                        count_item_scope5: 1 徽章
                        count_item_scope5: 3 大學
                        count_item_scope5: 5 校系排名
                        """
                        if count_item_scope5 == 1:
                            img_tag = item_scope5.find('img')
                            # print(img_tag)
                            if img_tag is not None:
                                img_tag_ = True
                            else:
                                img_tag_ = False
                            select.append(img_tag_)
                            
                        elif count_item_scope5 == 3:
                            # We'll drop later to suit up with img_tag_
                            if item_scope5.text != "":
                                department_name.append(item_scope5.text)
                            else:
                                # select.pop()
                                department_name.append(np.nan)
                            
                        elif count_item_scope5 == 5:
                            # to get the rank of each dept
                            try:
                                # img_tag = item_scope5.find('img')
                                # base64_str = item.find_all('img')[0]['src'].split(',')[1]
                                base64_str = item_scope5.find_all('img')[0]['src'].split(',')[1]
                            except IndexError:
                                dept_rank.append("None")
                            else:
                                
                                dept_rank_whole = ''
                                img_chi = ''
                                img_digit = ''
                                img_digit_2 = ''
                                img_digit_3 = ''
                                with open(r'data/dept_rank_crop.png', "wb") as fh:
                                    fh.write(base64.decodebytes(bytes(base64_str, 'utf-8')))
                                
                                img_dept_rank_crop = cv2.imread(r'data/dept_rank_crop.png')
                                # print(img_dept_rank_crop.shape)
                                if img_dept_rank_crop.shape[0] < 51:
                                    if img_dept_rank_crop.shape[1] < 100:
                                        # only one digit
                                        
                                        crop_img_chi = img_dept_rank_crop[0: 48, 0:48]
                                        cv2.imwrite(f'data/dept_crop_img_chi.png', crop_img_chi)
                                        path_to_image = f'data/dept_crop_img_chi.png'
                                        cv2.imwrite(f'data/dept_crop_img_chi.png', thresholding_(path_to_image))
                                        img_chi = pytesseract.image_to_string(Image.open(path_to_image), lang='chi_sim', config = '--oem 0 --psm 7')
                                        
                                        # first digit
                                        crop_img_digit = img_dept_rank_crop[0:48, 48:76]
                                        cv2.imwrite(f'data/dept_crop_img_digit.png', crop_img_digit)
                                        path_to_image = f'data/dept_crop_img_digit.png'
                                        cv2.imwrite(f'data/dept_crop_img_digit.png', thresholding_(path_to_image))
                                        img_digit = pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config = '--oem 1 --psm 7')
                                        
                                        img_chi = img_chi.strip()
                                        img_digit = img_digit.strip()
                                        dept_rank_whole = f'{img_chi}{img_digit}'
                                        dept_rank.append(dept_rank_whole)
                                        
                                    elif img_dept_rank_crop.shape[1] > 100 and img_dept_rank_crop.shape[1] < 120:
                                        # two digits
                                        
                                        crop_img_chi = img_dept_rank_crop[0: 48, 0:48]
                                        cv2.imwrite(f'data/dept_crop_img_chi.png', crop_img_chi)
                                        path_to_image = f'data/dept_crop_img_chi.png'
                                        cv2.imwrite(f'data/dept_crop_img_chi.png', thresholding_(path_to_image))
                                        img_chi = pytesseract.image_to_string(Image.open(path_to_image), lang='chi_sim', config = '--oem 0 --psm 7')
                                        
                                        # first digit
                                        crop_img_digit = img_dept_rank_crop[0:48, 48:76]
                                        cv2.imwrite(f'data/dept_crop_img_digit.png', crop_img_digit)
                                        path_to_image = f'data/dept_crop_img_digit.png'
                                        cv2.imwrite(f'data/dept_crop_img_digit.png', thresholding_(path_to_image))
                                        img_digit = pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config = '--oem 1 --psm 7')
                                        
                                        # second digit
                                        crop_img_digit = img_dept_rank_crop[0:48, 76:104]
                                        cv2.imwrite(f'data/dept_crop_img_digit_2.png', crop_img_digit)
                                        path_to_image = f'data/dept_crop_img_digit_2.png'
                                        cv2.imwrite(f'data/dept_crop_img_digit_2.png', thresholding_(path_to_image))
                                        img_digit_2 = pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config = '--oem 1 --psm 7')
                                        
                                        img_chi = img_chi.strip()
                                        img_digit = img_digit.strip()
                                        img_digit_2 = img_digit_2.strip()
                                        dept_rank_whole = f'{img_chi}{img_digit}{img_digit_2}'
                                        dept_rank.append(dept_rank_whole)
                                    
                                    elif img_dept_rank_crop.shape[1] > 120:
                                        # three digits
                                        
                                        crop_img_chi = img_dept_rank_crop[0: 48, 0:48]
                                        cv2.imwrite(f'data/dept_crop_img_chi.png', crop_img_chi)
                                        path_to_image = f'data/dept_crop_img_chi.png'
                                        cv2.imwrite(f'data/dept_crop_img_chi.png', thresholding_(path_to_image))
                                        img_chi = pytesseract.image_to_string(Image.open(path_to_image), lang='chi_sim', config = '--oem 0 --psm 7')
                                        
                                        # first digit
                                        crop_img_digit = img_dept_rank_crop[0:48, 48:76]
                                        cv2.imwrite(f'data/dept_crop_img_digit.png', crop_img_digit)
                                        path_to_image = f'data/dept_crop_img_digit.png'
                                        cv2.imwrite(f'data/dept_crop_img_digit.png', thresholding_(path_to_image))
                                        img_digit = pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config = '--oem 1 --psm 7')
                                        
                                        # second digit
                                        crop_img_digit = img_dept_rank_crop[0:48, 76:104]
                                        cv2.imwrite(f'data/dept_crop_img_digit_2.png', crop_img_digit)
                                        path_to_image = f'data/dept_crop_img_digit_2.png'
                                        cv2.imwrite(f'data/dept_crop_img_digit_2.png', thresholding_(path_to_image))
                                        img_digit_2 = pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config = '--oem 1 --psm 7')
                                        
                                        # third digit
                                        crop_img_digit = img_dept_rank_crop[0:48, 104:132]
                                        cv2.imwrite(f'data/dept_crop_img_digit_3.png', crop_img_digit)
                                        path_to_image = f'data/dept_crop_img_digit_3.png'
                                        cv2.imwrite(f'data/dept_crop_img_digit_3.png', thresholding_(path_to_image))
                                        img_digit_3 = pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config = '--oem 1 --psm 7')
                                        
                                        img_chi = img_chi.strip()
                                        img_digit = img_digit.strip()
                                        img_digit_2 = img_digit_2.strip()
                                        img_digit_3 = img_digit_3.strip()
                                        dept_rank_whole = f'{img_chi}{img_digit}{img_digit_2}{img_digit_3}'
                                        dept_rank.append(dept_rank_whole)
                                        
                                elif img_dept_rank_crop.shape[0] > 51:
                                    if img_dept_rank_crop.shape[1] < 100:
                                        # only one digit
                                        
                                        crop_img_chi = img_dept_rank_crop[0: 52, 0:48]
                                        cv2.imwrite(f'data/dept_crop_img_chi.png', crop_img_chi)
                                        path_to_image = f'data/dept_crop_img_chi.png'
                                        cv2.imwrite(f'data/dept_crop_img_chi.png', thresholding_(path_to_image))
                                        img_chi = pytesseract.image_to_string(Image.open(path_to_image), lang='chi_tra', config = '--oem 0 --psm 7')
                                        
                                        # first digit
                                        crop_img_digit = img_dept_rank_crop[0:52, 48:76]
                                        cv2.imwrite(f'data/dept_crop_img_digit.png', crop_img_digit)
                                        path_to_image = f'data/dept_crop_img_digit.png'
                                        cv2.imwrite(f'data/dept_crop_img_digit.png', thresholding_(path_to_image))
                                        img_digit = pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config = '--oem 1 --psm 7')
                                        
                                        img_chi = img_chi.strip()
                                        img_digit = img_digit.strip()
                                        dept_rank_whole = f'{img_chi}{img_digit}'
                                        dept_rank.append(dept_rank_whole)
                                        
                                    elif img_dept_rank_crop.shape[1] > 100 and img_dept_rank_crop.shape[1] < 120:
                                        # two digits
                                        
                                        crop_img_chi = img_dept_rank_crop[0: 52, 0:48]
                                        cv2.imwrite(f'data/dept_crop_img_chi.png', crop_img_chi)
                                        path_to_image = f'data/dept_crop_img_chi.png'
                                        cv2.imwrite(f'data/dept_crop_img_chi.png', thresholding_(path_to_image))
                                        img_chi = pytesseract.image_to_string(Image.open(path_to_image), lang='chi_tra', config = '--oem 0 --psm 7')
                                        
                                        # first digit
                                        crop_img_digit = img_dept_rank_crop[0:52, 48:76]
                                        cv2.imwrite(f'data/dept_crop_img_digit.png', crop_img_digit)
                                        path_to_image = f'data/dept_crop_img_digit.png'
                                        cv2.imwrite(f'data/dept_crop_img_digit.png', thresholding_(path_to_image))
                                        img_digit = pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config = '--oem 1 --psm 7')
                                        
                                        # second digit
                                        crop_img_digit = img_dept_rank_crop[0:52, 76:104]
                                        cv2.imwrite(f'data/dept_crop_img_digit.png', crop_img_digit)
                                        path_to_image = f'data/dept_crop_img_digit_2.png'
                                        cv2.imwrite(f'data/dept_crop_img_digi_2t.png', thresholding_(path_to_image))
                                        img_digit_2 = pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config = '--oem 1 --psm 7')
                                        
                                        img_chi = img_chi.strip()
                                        img_digit = img_digit.strip()
                                        img_digit_2 = img_digit_2.strip()
                                        dept_rank_whole = f'{img_chi}{img_digit}{img_digit_2}'
                                        dept_rank.append(dept_rank_whole)
                                    
                                    elif img_dept_rank_crop.shape[1] > 120:
                                        # three digits
                                        
                                        crop_img_chi = img_dept_rank_crop[0: 52, 0:48]
                                        cv2.imwrite(f'data/dept_crop_img_chi.png', crop_img_chi)
                                        path_to_image = f'data/dept_crop_img_chi.png'
                                        cv2.imwrite(f'data/dept_crop_img_chi.png', thresholding_(path_to_image))
                                        img_chi = pytesseract.image_to_string(Image.open(path_to_image), lang='chi_tra', config = '--oem 0 --psm 7')
                                        
                                        # first digit
                                        crop_img_digit = img_dept_rank_crop[0:52, 48:76]
                                        cv2.imwrite(f'data/dept_crop_img_digit.png', crop_img_digit)
                                        path_to_image = f'data/dept_crop_img_digit.png'
                                        cv2.imwrite(f'data/dept_crop_img_digit.png', thresholding_(path_to_image))
                                        img_digit = pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config = '--oem 1 --psm 7')
                                        
                                        # second digit
                                        crop_img_digit = img_dept_rank_crop[0:52, 76:104]
                                        cv2.imwrite(f'data/dept_crop_img_digit_2.png', crop_img_digit)
                                        path_to_image = f'data/dept_crop_img_digit_2.png'
                                        cv2.imwrite(f'data/dept_crop_img_digit_2.png', thresholding_(path_to_image))
                                        img_digit_2 = pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config = '--oem 1 --psm 7')
                                        
                                        # third digit
                                        crop_img_digit = img_dept_rank_crop[0:52, 104:132]
                                        cv2.imwrite(f'data/dept_crop_img_digit_3.png', crop_img_digit)
                                        path_to_image = f'data/dept_crop_img_digit_3.png'
                                        cv2.imwrite(f'data/dept_crop_img_digit_3.png', thresholding_(path_to_image))
                                        img_digit_3 = pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config = '--oem 1 --psm 7')
                                        
                                        img_chi = img_chi.strip()
                                        img_digit = img_digit.strip()
                                        img_digit_2 = img_digit_2.strip()
                                        img_digit_3 = img_digit_3.strip()
                                        dept_rank_whole = f'{img_chi}{img_digit}{img_digit_2}{img_digit_3}'
                                        dept_rank.append(dept_rank_whole)
                                        
                                # print(dept_rank_whole)
                                
                        count_item_scope5 += 1
                
            count_item += 1
        
        dict = {'rank': rank, 'exam_location': exam_location, 'stuno': stuno, 'univ': univ, 'year': year, 'department_name': department_name, 'dept_rank': dept_rank, 'select': select}
        df = pd.concat([df, pd.DataFrame([dict])], ignore_index = True)
    
    return df

def cross_interleave(df1, df2):
    # make two df cross-interleave
    df = pd.DataFrame()
    max_len = max(len(df1), len(df2))
    for line in range(max_len):
        if line < len(df1):
            df = pd.concat([df, df1.iloc[[line]]], ignore_index=True)
        if line < len(df2):
            df = pd.concat([df, df2.iloc[[line]]], ignore_index=True)
    return df

#### Function

In [4]:
def img_detect(item_scope2, img, langage, oem_psm):
    try:
        base64_str = item_scope2.find_all('img')[0]['src'].split(',')[1]
    except IndexError:
        return "None"
    else:
        with open(img, "wb") as fh:
            fh.write(base64.decodebytes(bytes(base64_str, 'utf-8')))
        path_to_image = img
        cv2.imwrite(img, thresholding_(path_to_image))
        return pytesseract.image_to_string(Image.open(path_to_image), lang=langage, config=oem_psm)
    
def img_detect_crop(img_, i):
    # start from (4, 20)
    basic_width = 4
    # 9 pixels each
    digit_width = 9
    
    crop_initial = digit_width*(i-1)+basic_width
    crop_end = digit_width*i+basic_width
    crop_img = img_[0: 20, crop_initial:crop_end]
    cv2.imwrite(f'data/crop_img{i}.png', crop_img)
    image = f'data/crop_img{i}.png'
    path_to_image = image
    cv2.imwrite(f'data/crop_img{i}.png', thresholding_(image))
    return pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config = '--oem 1 --psm 7')


def dept_img_detect_crop(img, i):
    # start from (4, 20)
    basic_width = 48
    # 9 pixels each
    digit_width = 28
    
    crop_initial = digit_width*(i-1)+basic_width
    crop_end = digit_width*i+basic_width
    crop_img = img[0: 48, crop_initial:crop_end]
    cv2.imwrite(f'data/dept_crop_img{i}.png', crop_img)
    image = f'data/dept_crop_img{i}.png'
    path_to_image = image
    cv2.imwrite(f'data/dept_crop_img{i}.png', thresholding_(image))
    return pytesseract.image_to_string(Image.open(path_to_image), lang='eng', config = '--oem 1 --psm 7')

##### Function

In [5]:
def thresholding_(png):
    image = cv2.imread(png)
    gray = get_grayscale(image)
    thresh_ = thresholding(gray)
    return thresh_

###### Function

In [6]:
# Convert the image to grayscale
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
 
# Thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]